# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [58]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import csv
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import json

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [23]:
data = pd.read_csv("weatherpy.csv", sep=",")
data.head(10)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,Ushuaia,-54.80,-68.30,44.60,56,40,25.28,AR
1,1,Oktyabr'skiy,54.48,53.47,45.39,92,53,5.77,RU
2,2,Severo-Kuril'sk,50.68,156.12,52.20,78,9,15.10,RU
3,3,Sovetskiy,61.36,63.58,51.08,89,100,8.50,RU
4,4,Kushima,31.46,131.23,73.40,88,90,3.36,JP
5,5,Hobart,-42.88,147.33,55.00,54,40,17.22,AU
6,6,Yellowknife,62.46,-114.35,48.20,49,75,14.99,CA
7,7,Meulaboh,4.14,96.13,76.64,88,92,4.09,ID
8,8,Bluff,-46.60,168.33,46.00,79,67,7.00,NZ
9,9,Abalak,58.13,68.59,48.69,92,94,3.27,RU


In [24]:
data.dropna(inplace=True)
data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,Ushuaia,-54.80,-68.30,44.60,56,40,25.28,AR
1,1,Oktyabr'skiy,54.48,53.47,45.39,92,53,5.77,RU
2,2,Severo-Kuril'sk,50.68,156.12,52.20,78,9,15.10,RU
3,3,Sovetskiy,61.36,63.58,51.08,89,100,8.50,RU
4,4,Kushima,31.46,131.23,73.40,88,90,3.36,JP


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
locations = data[["Lat", "Lng"]].astype(float)
humidity = data["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max(data["Humidity"]),
                                point_radius=2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [51]:
#1st cond: 60 < temp < 70
#2nd cond: wind < 5
#3rd cond: no clouds
new_data = data.loc[(data["Max Temp"] < 70) & (data["Max Temp"] > 60) & (data["Wind Speed"] < 5) 
                    & (data["Cloudiness"] == 0)]

new_data = new_data.dropna()

new_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
166,166,Ondjiva,-17.07,15.73,67.35,23,0,4.94,AO
216,216,Beshpagir,45.02,42.38,60.80,41,0,3.69,RU
249,249,Santa Fe,-31.63,-60.70,66.00,26,0,3.00,AR
299,299,Masasi,-10.72,38.80,67.14,89,0,4.47,TZ
414,414,Oktyabr'skoye,43.06,44.74,60.80,87,0,4.47,RU


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [60]:
hotel_df = new_data

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    print(f"Searching within {row['City']}...")
    
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    response = requests.get(base_url, params=params).json()

Searching within Ondjiva...
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -17.056641,
                    "lng": 15.6961799
                },
                "viewport": {
                    "northeast": {
                        "lat": -17.05525712010728,
                        "lng": 15.69749137989272
                    },
                    "southwest": {
                        "lat": -17.05795677989272,
                        "lng": 15.69479172010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "name": "\u00c1guia Verde Hotel",
            "photos": [
                {
                    "height": 2876,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/10378713988

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -10.7235668,
                    "lng": 38.8089164
                },
                "viewport": {
                    "northeast": {
                        "lat": -10.72231407010728,
                        "lng": 38.81031347989272
                    },
                    "southwest": {
                        "lat": -10.72501372989272,
                        "lng": 38.80761382010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "name": "Rose Royal Hotel",
            "photos": [
                {
                    "height": 3024,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/103049027595085571709\">Geology Li</a>"
    

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
